## Paper Text Mining
Here we are going to mine and process the text for a specific paper; in the end we are going to use three differents keyword extraction libraries to obtain all the main 

In [6]:
import os
import re
import shutil
from PyPDF2 import PdfFileReader

def extract_text(filename):
    #with open(pdf_path, 'rb') as f:
    f = open(filename, 'rb')
    pdf = PdfFileReader(f);
    if pdf.isEncrypted:
        try:
            pdf.decrypt('')
            print('File Decrypted (PyPDF2)')
        except:
            command = ("cp "+ filename +
                " temp.pdf; qpdf --password='' --decrypt temp.pdf " + filename
                + "; rm temp.pdf")
            os.system(command)
            print('File Decrypted (qpdf)')
            return ""
            #f = open(filename, 'rb')
            #pdf = PdfFileReader(f)
    else:
        print('File Not Encrypted')
    curtext= ""
    for i in range(0,pdf.getNumPages()):
        
        page = pdf.getPage(i)
        #print(page.extractText())
        curtext += page.extractText()
    return curtext
    
     
path = 'C:\\Users\\stefa\\OneDrive\\Desktop\\Uni\\Business\\Project\\Papers\\face_recognition_systems'
# path = 'C:\\Users\\stefa\\OneDrive\\Desktop\\Uni\\Business\\Project\\Papers\\fusion_method'
# path = 'C:\\Users\\stefa\\OneDrive\\Desktop\\Uni\\Business\\Project\\Papers\\keypoint_heatmap'
# path = 'C:\\Users\\stefa\\OneDrive\\Desktop\\Uni\\Business\\Project\\Papers\\face_cluster'
# path = 'C:\\Users\\stefa\\OneDrive\\Desktop\\Uni\\Business\\Project\\Papers\\neural_aggregation_network'
# path = 'C:\\Users\\stefa\\OneDrive\\Desktop\\Uni\\Business\\Project\\Papers\\field_view_head_mounted_display'
# path = 'C:\\Users\\stefa\\OneDrive\\Desktop\\Uni\\Business\\Project\\Papers\\feature_network'

print("Getting files...")
# getting all files from the directory given by the path
files = os.listdir(path)
# moving to the desired directory
os.chdir(path)
text = ""
cur_text = ""
filename = 'C:\\Users\\stefa\\OneDrive\\Desktop\\Uni\\Business\\Project\\Papers\\face_recognition_systems\\1-s2.0-S0031320399001399-main.pdf'
if filename.endswith('.pdf'):
    text += extract_text(filename); # read file content

# lower() is a Python function for strings
lower_text = ""
for pdf_text in text:
    lower_text += pdf_text.lower() #we pick each word and add to a variable, which will contain all the text

#white space removal 
def remove_whitespace(text):
    return  " ".join(text.split())

lower_text = remove_whitespace(lower_text)


#punctuation and digits removal: we replace any undesired character with a ''
for char in '?.,¡!/;:*#-~+±–—=%<>{}[]()@1234567890':  
    lower_text = lower_text.replace(char,'')

import pke

# initialize keyphrase extraction model, here TopicRank
print("Initializing extractor...")
extractor = pke.unsupervised.TopicRank()

# load the content of the document, here document is expected to be in raw
# format (i.e. a simple text file) and preprocessing is carried out using spacy
print("Loading text...");
extractor.load_document(input=lower_text, language='en')

# keyphrase candidate selection, in the case of TopicRank: sequences of nouns
# and adjectives (i.e. `(Noun|Adj)*`)
print("Candidate Selection...")
extractor.candidate_selection()

# candidate weighting, in the case of TopicRank: using a random walk algorithm
print("Weighting...")
extractor.candidate_weighting()

# N-best selection, keyphrases contains the 10 highest scored candidates as
# (keyphrase, score) tuples
print("Selecting 10 best candidates...")
keyphrases = extractor.get_n_best(n=10)
for tuple in keyphrases:
    print(tuple[0])
    lower_text = lower_text.replace(tuple[0],'')
print(keyphrases)

from keybert import KeyBERT
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(lower_text)
for tuple in keywords:
    print(tuple[0])
    lower_text = lower_text.replace(tuple[0],'')

print(kw_model.extract_keywords(lower_text, keyphrase_ngram_range=(1, 1), stop_words=None))

import nltk
nltk.sent_tokenize(lower_text)

# the output is a list, where each element is a token of the original text
tokenized_text = nltk.word_tokenize(lower_text)

from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

# we prepare a empty list, which will contain the words after the stopwords removal
tokenized_vector = []

# we iterate into the list of tokens obtained through the tokenization
for token in tokenized_text:
    # if a token is not a stopword, we insert it in the list
    if token not in stopwords_en:
        tokenized_vector.append(token)

# the output is a list of all the tokens of the original text excluding the stopwords

pos_tagging = nltk.pos_tag(tokenized_vector)

cleaned_POS_text = []

for tuple in pos_tagging:
    # POS tagged text is a list of tuples, where the first element tuple[0] is a token and the second one tuple[1] is
    # the Part of Speech. If the POS has length == 1, the token is punctuation, otherwise it is not, and we insert it
    # in the list cleaned_POS_text
    if len(tuple[1]) > 1:
        cleaned_POS_text.append(tuple)

def simpler_pos_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return "a"
    elif nltk_tag.startswith('V'):
        return "v"
    elif nltk_tag.startswith('N'):
        return "n"
    elif nltk_tag.startswith('R'):
        return "r"
    else:         
        return None
    
simpler_POS_text = []

# for each tuple of the list, we create a new tuple: the first element is the token, the second is
# the simplified pos tag, obtained calling the function simpler_pos_tag()
# then we append the new created tuple to a new list, which will be the output
for tuple in cleaned_POS_text:
    if tuple[1] == 'NNP':   #this is because there is some text in japanese categorized as 'NNP';
                            #no other relevant words are categorized in such a way
        continue;
    POS_tuple = (tuple[0], simpler_pos_tag(tuple[1]))
    simpler_POS_text.append(POS_tuple)
    

from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

lemmatized_text = []

for tuple in simpler_POS_text:
    if (tuple[1] == None):
        lemmatized_text.append(lemmatizer.lemmatize(tuple[0]))
    else:
        lemmatized_text.append(lemmatizer.lemmatize(tuple[0], pos=tuple[1]))

lem_text = ""
for abstract_text in lemmatized_text:
    lem_text += abstract_text + " " #we pick each word and add to a variable, which will contain all the text
lem_text

import pke
# initialize keyphrase extraction model, here TopicRank
print("Initializing extractor...")
extractor = pke.unsupervised.TopicRank()

# load the content of the document, here document is expected to be in raw
# format (i.e. a simple text file) and preprocessing is carried out using spacy
print("Loading text...");
extractor.load_document(input=lem_text, language='en')

# keyphrase candidate selection, in the case of TopicRank: sequences of nouns
# and adjectives (i.e. `(Noun|Adj)*`)
print("Candidate Selection...")
extractor.candidate_selection()

# candidate weighting, in the case of TopicRank: using a random walk algorithm
print("Weighting...")
extractor.candidate_weighting()

# N-best selection, keyphrases contains the 10 highest scored candidates as
# (keyphrase, score) tuples
print("Selecting 40 best candidates...")
keyphrases = extractor.get_n_best(n=80)
for tuple in keyphrases:
    print(tuple[0])
#print(keyphrases)

from keybert import KeyBERT
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(lem_text)
for tuple in keywords:
    print(tuple[0])

print(kw_model.extract_keywords(lem_text, keyphrase_ngram_range=(1, 1), stop_words=None))

from rake_nltk import Rake

# Uses stopwords for english from NLTK, and all puntuation characters by
# default
r = Rake()

# Extraction given the text.
r.extract_keywords_from_text(lem_text)

# To get keyword phrases ranked highest to lowest.
r.get_ranked_phrases()

# To get keyword phrases ranked highest to lowest with scores.
r.get_ranked_phrases_with_scores()



Getting files...
File Not Encrypted


C:\Users\stefa\anaconda3\lib\site-packages\PyPDF2\_reader.py:1351: PdfReadWarning: Xref table not zero-indexed. ID numbers for objects will be corrected.
  warnings.warn(


Initializing extractor...
Loading text...
Candidate Selection...
Weighting...
Selecting 10 best candidates...
sample images
class containsmsamples
optimal discriminant vectors
fig
great number
set
face image
recognition society
dierent methods
withinclass scatter matrix
[('sample images', 0.03142200965534665), ('class containsmsamples', 0.02467444277138319), ('optimal discriminant vectors', 0.021151517342831692), ('fig', 0.018339550083292973), ('great number', 0.01739710648680436), ('set', 0.017002328074481318), ('face image', 0.01578486388962984), ('recognition society', 0.015242317449343533), ('dierent methods', 0.014175236730822818), ('withinclass scatter matrix', 0.014095380313097733)]
discriminantvectors
lda
discriminantvector
recognition
discriminant
[('eigenfaces', 0.3929), ('keywordsface', 0.3597), ('featurevectors', 0.3572), ('pca', 0.325), ('faces', 0.3192)]
Initializing extractor...
Loading text...
Candidate Selection...
Weighting...
Selecting 40 best candidates...
highdimen

[(1292269.3173891134,
  "rnfkq eigenvectors corre sponding large eigenvalue matrix sbswsb small sample size problem occur point eigenvectors sbswsbwill di cult compute due singularity problem order avoid direct computation sbswsb liu et al suggest derive dis criminant vector complementary subspace null space ofstst '' sbsw denote total scat ter matrix null space ofstis span eigenvectors correspond zero eigenvalue ofst since total scatter matrixstin complementary subspace nonsingular feasible follow normal process derive vector subspace however still critical problem associate approach '' rst problem liu et al approach validity vector problem know purpose maximize betweenclass scatter minimize thelf chen et alpattern withinclass scatter simultaneously special case whereqtswq '' andqtsbqo eq de '' nitely reach maximum value offkq however arbitrary projection vectorqsatisfying condition guarantee derivation maximumqtsbqvalue circumstance correct process complete withinclass scatter minimi